# Chapter 8. Higher-order functions

- Function types allow you to declare a variable, parameter, or function return value that holds a reference to a function.
- Higher-order functions (HOFs) help structure code and remove duplication
 - functions that take in other functions as arguments or returns them
 - create them by using a function type as the type of a function parameter or return value
 - help create reusable, flexible code
- Inline functions 
 - remove the performance costs of lambdas by being inserted directly into the code of the calling function upon compilation, so it's as if you wrote it there directly
- Non-local returns and labels
 - lambdas return from enclosing function by default, but can return from anywhere if you use labels
- Anonymous functions - alternative syntax to lambdas, return locally
 - use when you need to write a block of code with multiple exit points

## 8.1. DECLARING HIGHER-ORDER FUNCTIONS

What is a HOF?
- a function that takes another function as an argument or returns one
- that function can be represented by lambdas or function references

### 8.1.1 How to declare a function

- `() -> Unit = { ... }`
- `(Int, String) -> Unit = { x, y -> ... } `
- `(String, String) -> String?` <- can be nullable

In [1]:
// a regular function type
val sum: (Int, Int) -> Int = { x, y -> x + y }

// return types can be nullable
val maybeNull: (Int, Int) -> Int? = { x, y -> if (x + y < 0) null else x + y }

// if the function itself is nullable, you'll need parentheses around whole thing
var funOrNull: ((Int, Int) -> Int)? = null

// higher-order-function
fun printResult(x: Int, y: Int, fn: (Int, Int) -> Any?) = println(fn(x, y))

printResult(1, 2, sum) 
printResult(-4, -3, maybeNull)
printResult(2, 3) { x, y -> "anonymous lambda called with: $x, $y" }

3
null
anonymous lambda called with: 2, 3


### 8.1.2. Calling functions passed as arguments

- Same as regular functions

In [9]:
// filter for only strings that match a predicate
fun List<String>.filter(predicate: (String) -> Boolean): List<String> {
    val filteredList: MutableList<String> = mutableListOf()
    for (word in this) {
        if (predicate(word)) filteredList.add(word)
    }
    return filteredList.toList()
}

println(listOf("do", "cat", "four").filter { word -> word.length > 2  })

[cat, four]


### 8.1.3. Using function types from Java

Under the hood - a variable of function type is instance of class implementing a FunctionN interface where N = number of arguments:
- Function0<R>, Function1<P1, R> etc
- Each function has a single `invoke`. 


In [22]:
/* Kotlin: Given this function in Kotlin... */
fun logIt(fn: (Int) -> Int) { println(fn(42)) }

/**
* Java (pre-8) - pass in an anonymous function that overrides invoke
logIt(new Function<Integer, Integer>() {
    @Override
    public Integer invoke(Integer number) {
        return number + 1;
    }
}
**/

/**
Note that if your Kotlin function returns Unit, like (String) -> Unit
you need to return Unit.INSTANCE in Java, *not* void

List<String> strings = listOf("42")
CollectionsKt.forEach(strings, { s -> {
    println(s)
    return Unit.INSTANCE
}})
**/

### 8.1.4. Default and null values for parameters with function types

In [6]:
// Approach 1: use a default function for your function params

fun <T> Collection<T>.joinToString(
    separator:String = ", ",
    transform: (T) -> String = { it.toString().toUpperCase() }
): String {
    val result = StringBuilder()
    for ((idx, el) in this.withIndex()) {
        if (idx > 0) result.append(separator)
        result.append(transform(el))
    }
    return result.toString()
}

val letters = listOf("cats", "dogs")
println(letters.joinToString())

CATS, DOGS


In [8]:
// Approach 2: make your function param nullable

fun <T> Collection<T>.joinToString(
    separator:String = ", ",
    transform: ((T) -> String)? = null
): String {
    val result = StringBuilder()
    for ((idx, el) in this.withIndex()) {
        if (idx > 0) result.append(separator)
        result.append(transform?.invoke(el) ?: el.toString().toUpperCase())
    }
    return result.toString()
}

val letters = listOf("cats", "dogs")
println(letters.joinToString())

CATS, DOGS


### 8.1.5. Returning functions from functions

Not as common as function params, but cool for currying!


In [2]:
// simple curry example

// a function that returns another function that adds some base num
fun addWithBase(base: Int): (Int) -> Int { 
    return { anotherNum -> (base + anotherNum) }
}

val addOne = addWithBase(1) 
println(addOne(41))
println(addWithBase(2)(4))

42
6


In [3]:
// Note that in current Kotlin versions, don't need to nest these classes inside the sealed class
sealed class Block {
    data class TextBlock(val text: String) : Block()
    data class ImageBlock(val url: String) : Block()
}

// each block type has its own "is valid" predicate
fun <T : Block> getBlockValidator(clazz: Class<T>): (T) -> Boolean {
    when(clazz){
        Block.TextBlock::class.java -> return { 
            b: Block -> b is Block.TextBlock && b.text.isNotBlank()
        }
        Block.ImageBlock::class.java -> return { 
            b: Block -> b is Block.ImageBlock && b.url.endsWith(".jpg")
        }
        else -> return { b: Block -> false }
    }
}

val TEXT_BLOCK_VALIDATOR = getBlockValidator(Block.TextBlock::class.java)
val IMAGE_BLOCK_VALIDATOR = getBlockValidator(Block.ImageBlock::class.java)

println(TEXT_BLOCK_VALIDATOR(Block.TextBlock(""))) // false
println(TEXT_BLOCK_VALIDATOR(Block.TextBlock("coalesce"))) // true
println(IMAGE_BLOCK_VALIDATOR(Block.ImageBlock("shook"))) // false
println(IMAGE_BLOCK_VALIDATOR(Block.ImageBlock("shook.jpg"))) // true


false
true
false
true


### 8.1.6. Removing duplication through lambdas

Function types and lambda expressions => reusable code.
Strategy pattern - use a general function type as strategy, and pass different lambda expressions as different strategies

In [ ]:
enum class PostAction { LIKE, REBLOG, LIGHTBOX, NOTES_CLICK }
enum class NavigationState { DASHBOARD, SEARCH, BLOG, UNKNOWN }

data class PostEvent(
    val action: PostAction,
    val screen: NavigationState = NavigationState.UNKNOWN
)

fun List<PostEvent>.logPostActions(predicate: (PostEvent) -> Boolean) = 
    filter(predicate).map { "{it.action} on {it.screen}" }

val postEvents = listOf(
    PostEvent(PostAction.LIKE, NavigationState.DASHBOARD),
    PostEvent(PostAction.REBLOG, NavigationState.BLOG),
    PostEvent(PostAction.NOTES_CLICK),
    PostEvent(PostAction.LIGHTBOX),
    PostEvent(PostAction.REBLOG, NavigationState.SEARCH)
)
// filter for events on blog screen
val blogScreenEvents = postEvents.logPostActions {
    it.screen == NavigationState.BLOG
}
// filter for reblog events on any scren
val reblogEvents = postEvents.logPostActions {
    it.action == PostAction.REBLOG
}
println(unknownScreenEvents)
println(reblogEvents)

## 8.2. INLINE FUNCTIONS: REMOVING THE OVERHEAD OF LAMBDAS

Do lambdas present some performance problems?

Lambdas are normally compiled to anonymous classes. Which means:
 - every time you use a lambda expression, an extra class is created (so at least 1 extra allocation)
 - if it captures some variables, then a new object is created on every invocation (so N extra allocations, where N = number of invocations)
 
[Like in Java, what happens in Kotlin varies in different cases](https://discuss.kotlinlang.org/t/lambdas-and-implicit-references-to-the-instance-of-the-enclosing-class/2288/2):
- If the lambda is passed to an inline function and isn’t marked noinline, then the whole thing boils away and no additional classes or objects are created.
- If the lambda doesn’t capture, then it’ll be emitted as a singleton class whose instance is reused again and again (one class+one object allocation).
- If the lambda captures then a new object is created each time the lambda is used.

Without `inline`:

```kotlin
fun higherOrderFunction(aLambda: () -> Unit) {
        doSomething()
        aLambda()
        doAnotherThing()
}
```

```java
// if you call this in a loop, you'll create N new function objects
public void callingFunction(){ 
    higherOrderFunction(new Function() {
        @Override
        public void invoke() {
          //aLambda's logic here.  

        }
    });
}
```

### 8.2.1. How inlining works

The body of the lambda expression that is passed as an argument is substituted directly into the resulting code:

```kotlin
inline fun higherOrderFunction(aLambda: () -> Unit) {
    doSomething()
    aLambda()
    doAnotherThing()
}
fun callFunction(msg: String) {
    higherOrderFunction{ println(msg) }
}
```

```java
public void callingFunction(msg) {
  doSomething()
  // inline lambda code
  System.out.println(msg)
  doAnotherThing()
}
```

In [1]:
// Inlined example

class Lock {
    fun lock() = println("locking...")
    fun unlock() = println("unlocking...")
}

// original inlined function
inline fun <T> synchronized(lock: Lock, action: () -> T): T {
    lock.lock()
    try {
        return action()
    }
    finally {
        lock.unlock()
    }
}

// now calling it in kotlin
fun foo(l: Lock) {
    println("=== Before inline sync call == ")
    
    synchronized(l) {
        println("ACTION")
    }
    
    println("=== After inline sync call == ")
}

// compiles to this in java
fun compiledFoo(l: Lock) {
    println("=== compiledFoo: Before inline sync call == ")

    /* inlined fn */
    l.lock()
    try {
        println("compiledFoo: ACTION")
    } finally {
        l.unlock()
    }
    /* end inlined fn */
    
    println("=== compiledFoo: After inline sync call == ")
}

foo(Lock())
compiledFoo(Lock())

=== Before inline sync call == 
locking...
ACTION
unlocking...
=== After inline sync call == 
=== compiledFoo: Before inline sync call == 
locking...
compiledFoo: ACTION
unlocking...
=== compiledFoo: After inline sync call == 


In [3]:
// if the lambda code isn't available at the site where 
// the inline function is called, it's not inlined

class LockOwner(val l: Lock) {
    fun runUnderLock(action: () -> Unit) {
        synchronized(l, action)
    }
    
    // compiles to this in java
    fun compiledRunUnderLock(action: () -> Unit) {
        l.lock()
        try {
            action() // not inlined because there's no lambda
        } finally {
            l.unlock()
        }
    }
}

LockOwner(Lock()).runUnderLock {
    println("hi i am a lock")
}

// If you have two uses of an inline function in different 
// locations with different lambdas, then every call site will be 
// inlined independently. 
// The code of the inline function will be copied to both locations 
// where you use it, with different lambdas substituted into it.

hi i am a lock


### 8.2.2. Restrictions on inline functions

Not every function that uses lambdas can be inlined, because the code is substituted directly in the resulting code:
- generally ok if it's called directly, or passed as an argument to another inline function
- but if the param is stored somewhere for later use, the code of the lambda expression can't be inlined because there must be an object that contains this code
 - compiler will complain `Illegal usage of inline-parameter.`

In [16]:
data class CustomSequence<T, R>(val transform: (T) -> R)

inline fun <T, R> Sequence<T>.map(transform: (T) -> R): CustomSequence<T, R> {
    // `transform` is stored as a property in TransformingSequence
    // so it has to use the standard non-inline representation
    // (an anonymous class implementing Function1 interface)
    return CustomSequence(transform)
}

error: illegal usage of inline-parameter 'transform' in 'public final inline fun <T, R> Sequence<T>.map(transform: (T) -> R): Line_16.CustomSequence<T, R> defined in Line_16'. Add 'noinline' modifier to the parameter declaration
    return CustomSequence(transform)
                          ^


#### Noinline and Crossinline

https://android.jlelse.eu/inline-noinline-crossinline-what-do-they-mean-b13f48e113c2

Noinline:
- used with functions that expect two or more lambdas as arguments, and you want to inline only some of them

Crossinline:
- used to mark lambdas that mustn’t allow non-local returns, especially when such lambda is passed to another execution context such as a higher order function that is not inlined, a local object or a nested function. In other words, you won’t be able to do a return in such lambdas.


In [2]:
// Noinline
inline fun foo(inlined: () -> Unit, noinline notInlined: () -> Unit) {
  // ...
}

In [6]:
// Crossinline

inline fun higherOrderFunction(crossinline aLambda: () -> Unit) {
    normalFunction {
        aLambda() //must mark aLambda as crossinline to use in this context.
    }
}

fun normalFunction(aLambda: () -> Unit) {
    println("normalFunction start---")
    aLambda()
    println("normalFunction end---")
    return
}

fun callingFunction() {
    higherOrderFunction { 
        println("lambda passed to higherOrderFunction")
        // return  // Error. Can't return from here.
    }
}

callingFunction()

normalFunction start---
lambda passed to higherOrderFunction
normalFunction end---


Interop:
- compiler fully supports inlining functions across modules and functions defined in third-party libraries.
- can also call most inline functions from Java (will be compiled as regular function calls)

### 8.2.3. Inlining collection operations

Most Kotlin collection functions take lambda args. Does that worsen perforance compared to implementing the operations directly?
- the standard functions like filter, map etc are `inline` so it should be ok!



https://github.com/JetBrains/kotlin/blob/master/libraries/stdlib/src/generated/_Collections.kt

```kotlin
/**
 * Returns a list containing only elements matching the given [predicate].
 */
inline fun <T> Iterable<T>.filter(predicate: (T) -> Boolean): List<T> {
    return filterTo(ArrayList<T>(), predicate)
}
```
```kotlin
/**
 * Returns a list containing the results of applying the given [transform] function
 * to each element in the original collection.
 */
inline fun <T, R> Iterable<T>.map(transform: (T) -> R): List<R> {
    return mapTo(ArrayList<R>(collectionSizeOrDefault(10)), transform)
}
```
```kotlin
/**
 * Returns the first element matching the given [predicate], or `null` if no such element was found.
 */
@kotlin.internal.InlineOnly
inline fun <T> Iterable<T>.find(predicate: (T) -> Boolean): T? {
    return firstOrNull(predicate)
}
```

In [10]:
// But what if you have both `filter` and `map`?

// regular lists calculate eagerly
val list = listOf(10, 12, 31, 1, 4)
        .filter { it % 2 == 0 } 
        // creates intermediate collection [10, 12, 4] ...
        .map { println("got: $it"); it }
       
println("BEFORE")
println(list)

got: 10
got: 12
got: 4
BEFORE
[10, 12, 4]


In [19]:
// If the number of elements is large you can use a sequence instead
// by adding an `asSequence` call to the chain:

// Note that WHERE you place the `asSequence` matters!
val sequence = listOf(10, 12, 31, 1, 4)
        .asSequence() // to run lazily
        .filter { it % 2 == 0 } 
        .map { println("got: $it"); it }
       // .asSequence() // putting it here doesn't work
        

println("BEFORE")
println(sequence)

// Only runs when you call a terminal operation on it
println(sequence.joinToString())

// `map` returns another Sequence
println(sequence.map { "mapped" })

BEFORE
kotlin.sequences.TransformingSequence@437674c
got: 10
got: 12
got: 4
10, 12, 4
kotlin.sequences.TransformingSequence@60f1d1cb


#### Why not sequence all the things?

Operations on sequences are lazy, BUT lambdas used to process a sequence aren’t inlined.
Each intermediate sequence is represented as an object storing a lambda in its field, and the terminal operation causes a chain of calls through each intermediate sequence to be performed.

[SO: "Kotlin's Iterable and Sequence look exactly same. Why are two types required?"](https://stackoverflow.com/questions/35629159/kotlins-iterable-and-sequence-look-exactly-same-why-are-two-types-required):

> "Lazy is usually less performant for smaller and more commonly used collections. A copy can be faster than a lazy eval if taking advantage of CPU cache etc."

> "A common pitfall with lazy evaluation is capturing the context and storing resulting lazy computation in a field along with all the captured locals and whatever they hold. Hence, hard to debug memory leaks."


### 8.2.4. Deciding when to declare functions as inline

Why not `inline` all the things?

Not a great idea - only likely useful with functions that take lambdas as arguments.

For regular function calls, the JVM already does a lot of inliing for you when translating bytecode -> machine code. In bytecode, the implementation of each function is repeated only once and doesn’t need to be copied to every place where the function is called, like with Kotlin’s `inline`, and the stacktrace is clearer if the function is called directly.

Why inline functions with lambda args?
* you avoid overhead - you save not only on the call, but also on the creation of the extra class for each lambda and an object for the lambda instance. 
* JVM isn’t smart enough to always perform inlining through the call and the lambda
* inlining lets you use features that are impossible to make work with regular lambdas, such as non-local returns

Even so, still some issues:
- Watch out for the code size:
 - If the function you want to inline is large, copying its bytecode into every call site could be expensive in terms of bytecode size. In that case, you should try to extract the code not related to the lambda arguments into a separate non-inline function.
- [With inline functions, you will not be able to access private members/methods of your enclosing class.](https://android.jlelse.eu/inline-noinline-crossinline-what-do-they-mean-b13f48e113c2)
    - Need to make those members/methods `internal` and then annotate them with @PublishedApi.


### 8.2.5. Using inlined lambdas for resource management

#### Resource Management

- acquiring a resource before an operation and releasing it afterwards
- resource = a file, a lock, a database transaction, etc
- replacement for `try/finally`


In [12]:
// EX: The use function - an inline extension function called on 
// a closable resource. Receives a lambda as an argument and automatically
// closes the resource after running the lambda.

interface Closeable {
    fun close() = println("=== closed! ===")
}

data class SomeCloseableResource(val name: String = "my resource") 
: Closeable

inline fun <T: Closeable> T.use(action: (T) -> Any?) {
    try {
        println("=== trying action ===")
        action(this) // any action like I/O, buffered reader etc
    } finally {
        this.close()
    }
}

// Use it!
SomeCloseableResource().use { res -> println("printing: ${res.name}") }

=== trying action ===
printing: my resource
=== closed! ===


## 8.3 CONTROL FLOW IN HIGHER-ORDER FUNCTIONS

**RULE: `return` returns from the closest function declared using the fun keyword.**

How does `return` work inside lambdas?
- lambdas don't use the `fun` keyword, so by default they return from the outer function!
- applicable only to `inline` lambdas, `non-inline` lambdas can't have `returns` in them
 - Why? Because they can save the lambda passed to it in a variable and execute it later when the function has already returned.
 
 
### 8.3.2. Returning from lambdas: return with a label

You can write a local return from a lambda expression as well:
- equivalent to `break` in a loop
- use `labels` after the `return`:
 - can be explicit label with custom name, or juse use the name of the function that takes this lambda (ex: `return@forEach`, `return@map`)
 - note you can't combine these (explicit `label` + label using function name), since lambdas can't have more than one label


In [1]:
// Labels are useful inside lambdas!

// Usually, lambdas return out of the enclosing function...
fun lookForFirstEven(nums: List<Int>) {
    nums.forEach{
        if (it % 2 == 0) {
            println("found: $it")
            return // will return out of `lookForFirstEven`
        }
    }
    println("couldn't find an even num") // this is always printed now
}

val list = listOf(3, 5, 11, 14, 2, 0)
lookForFirstEven(list)

found: 14


In [3]:
// But you can return at a label
fun lookForFirstEvenLocal(nums: List<Int>) {
    nums.forEach mylabel@{
        if (it % 2 == 0) {
            println("found: $it")
            return@mylabel // OR just return@forEach
        }
    }
    println("maybe had even num?") // this is always printed now
}

val list = listOf(3, 5, 11, 14, 2, 0)
lookForFirstEvenLocal(list)

found: 14
found: 2
found: 0
maybe had even num?


In [7]:
// Labeled "this" expressions

// Lambdas with receivers contain an implicit `context` that can be 
// accessed via a `this` reference in a lambda

println(StringBuilder().apply sb@{ // label the lambda `sb`
    listOf(1, 2, 3).apply {
        // `this@sb` is the outer `apply`
        // `this` is the closest implicit receiver (the local `apply`)
        this@sb.append(this.toString()) 
    }
})

println(StringBuilder().apply { // label the lambda `sb`
    listOf(1, 2, 3).run { // have to switch to `run` to use names
        this@apply.append(this.toString()) 
    }
})

[1, 2, 3]
[1, 2, 3]


### 8.3.3. Anonymous functions: local returns by default

*An alternative way to labeled returns to return locally.*

Remember the rule: `return` returns from the closest function declared using the fun keyword.
- Because anonymous functions use `fun`, `return` returns from the anonymous function, not the enclosing one.

Note anonymous functions are just another syntactic form of lambdas. They are NOT regular function declarations.


In [3]:
// Anonymous functions return locally by default
fun lookForFirstEvenAnonymous(nums: List<Int>) {
    nums.forEach(fun (num) {
        if (num % 2 == 0) {
            println("found: $num")
            return
        }
    })
    println("maybe had an even num?") // this is always printed now
} 

val list = listOf(3, 5, 11, 14, 2, 0)
lookForFirstEvenAnonymous(list)

// Anonymous functions have same rules as regular functions 
// for specifying return type

// Block bodies require explicit return types
println(list.filter(fun (num): Boolean {
    return num % 2 == 0
}))

// Expression bodies can omit the return type
println(list.filter(fun (num) = num % 2 == 0))


found: 14
found: 2
found: 0
maybe had an even num?
[14, 2, 0]
[14, 2, 0]


### Reading

https://android.jlelse.eu/inline-noinline-crossinline-what-do-they-mean-b13f48e113c2

